# Decorators

A decorator is just a function that takes in another function as an argument, adds some functionality to it (without modifying the actual function) and returns it with the added functionality.

For example, we'll create a decorator, called `timer`, that just times the execution of a function.

In [68]:
from time import time

In [81]:
def timer(func):
    def wrapper():
        start = time()
        result = func()
        end = time()
        print(f'{func.__name__} took {end - start:.4f} seconds to run.')
        return result
    return wrapper

`timer` takes in a function and returns a new function that just times how long it takes our function to execute and prints the results.

We'll define some function that takes some time to compute, then we can *decorate* it with our timer decorator to see how long it takes.

In [82]:
# waste_time just sums up all the numbers 
# between 1 and 10 million.
def waste_time():
    sum = 1
    for i in range(1, 10_000_001):
        sum += i  
    return sum

print('run waste_time once before we decorate it')
print(f'{waste_time():,}')

# To decorate this function with our timer we can just write:
waste_time = timer(waste_time)

run waste_time once before we decorate it
50,000,005,000,001


In [83]:
waste_time()

waste_time took 0.6890 seconds to run.


50000005000001

We see that in order to apply our decorator we have to call
```python
waste_time = timer(waste_time)
```
or more generally
```python
func = decorator(func)
```
but python also offers a different way of applying a decorator to a function, just add
```python 
@decorator
def func():
    ...
```

In [84]:
@timer
def waste_time():
    sum = 1
    for i in range(1, 10_000_001):
        sum += i  
    return sum

In [85]:
waste_time()

waste_time took 0.6864 seconds to run.


50000005000001

```python
@decorator
def func():
    ...
```
is just [syntactic sugar](https://en.wikipedia.org/wiki/Syntactic_sugar) for 
```python
func = decorator(func)
```